In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import glob
import os
import suspect
import multiprocessing as mp
plt.rcParams['figure.figsize'] = [10, 7]

In [ ]:
specs = glob.glob('/home/orco/data/MacroMols/MRS/MacroMols_3T/rawdata/sub-*/*rm-special.dat')
specs

In [ ]:
specs.pop(5)

In [ ]:
water = [i.replace('rm-special.dat','rm-special-water_woOVS.dat') for i in specs]
water

In [ ]:
[os.path.isfile(i) for i in water]

In [ ]:
%%time
specs_data = [suspect.io.load_twix(i) for i in specs]


In [ ]:
waters = [suspect.io.load_twix(i) for i in water]

In [ ]:
plt.plot(specs_data[0][0,0].frequency_axis_ppm(), specs_data[0][0,0].spectrum().real)
plt.plot(specs_data[0][1,0].frequency_axis_ppm(), specs_data[0][1,0].spectrum().real)
plt.xlim([5,0])

In [ ]:
def mean_everytwo(mm1):
    mm1_everytwo = []
    for i in range(0,mm1.shape[0]-1,2):
        mm1_everytwo.append((mm1[i]+mm1[i+1])/2)
    return mm1.inherit(np.array(mm1_everytwo))

In [ ]:
specs_data[0].shape

In [ ]:
def _avg_everytwo(arr):
    return 0.5*(arr[0::2,:, :] + arr[1::2,:, :])

In [ ]:
%%time
avg_everytwo_data = [_avg_everytwo(i) for i in specs_data]
avg_everytwo_water = [_avg_everytwo(i) for i in waters]

In [ ]:
#Rodgers, C. T., & Robson, M. D. (2010). Receive array magnetic resonance spectroscopy: Whitened singular value decomposition (WSVD) gives optimal Bayesian solution. Magnetic Resonance in Medicine, 63(4), 881–891
def coil_combine(data, wref, noise_points=None):
    if not noise_points:
        noise_points = int(data.shape[-1]/8)
    noise = data[:,:,-noise_points:]
    noise = np.moveaxis(noise, -2, 0).reshape((32, -1))
    white_data = suspect.processing.channel_combination.whiten(data, noise)
    white_wref = suspect.processing.channel_combination.whiten(wref, noise)
    noise = white_data[:, :, -noise_points:]
    channel_weights = suspect.processing.channel_combination.svd_weighting(np.mean(white_wref, axis=0))
    cc_data = suspect.processing.channel_combination.combine_channels(white_data, channel_weights)
    cc_wref = suspect.processing.channel_combination.combine_channels(white_wref, channel_weights)
    return cc_data, cc_wref

In [ ]:
%%time
cc_data = list()
cc_wref = list()
for n in range(len(avg_everytwo_data)):
    d,w = coil_combine(avg_everytwo_data[n],avg_everytwo_water[n])
    cc_data.append(d)
    cc_wref.append(w)

In [ ]:
spectra = cc_data[9].spectrum()
frequency_slice = spectra.slice_ppm(4.2, 0)
plt.imshow(spectra[:, frequency_slice].T.real, extent=[0, 32, 0, 4.2], aspect='auto')

In [ ]:
first = cc_wref[0][1]
plt.plot(first.frequency_axis_ppm(), first.spectrum().real)
plt.xlim([5,0])
plt.axvline(4.7)

In [ ]:
corr = suspect.processing.frequency_correction.residual_water_alignment(first)
first = first.adjust_frequency(-corr)
corr

In [ ]:
plt.plot(first.frequency_axis_ppm(), first.adjust_frequency(corr).spectrum().real)
plt.plot(first.frequency_axis_ppm(), first.adjust_frequency(0).spectrum().real)
plt.xlim([5,4])
plt.axvline(4.7)

In [ ]:
w_freq_corr  = [suspect.processing.frequency_correction.correct_frequency_and_phase(i,first) for i in cc_wref]

In [ ]:
first = cc_data[9][7]
plt.plot(first.frequency_axis_ppm(), first.adjust_phase(-0,).adjust_frequency(3).spectrum().real)
plt.xlim([7,0])
plt.axvline(1.99)
plt.axvline(4.7)
plt.axvline(3.027)
plt.axvline(0.94)
plt.axhline(0)

In [ ]:
first = first.adjust_frequency(3)

In [ ]:
freq_corr2 = [suspect.processing.frequency_correction.correct_frequency_and_phase(i,i[0]) for i in cc_data]

In [ ]:
sr_spectra = freq_corr2[2].spectrum()
frequency_slice = sr_spectra.slice_ppm(4.2, 0)
plt.imshow(sr_spectra[:, frequency_slice].T.real, extent=[0, 32, 0, 4.2], aspect='auto')

In [ ]:
figure, axis = plt.subplots(3,5)
for n,i in enumerate(freq_corr2):
    
    spectra = i.spectrum()
    frequency_slice = sr_spectra.slice_ppm(4.2, 0)
    axis[int(n/5),n%5].imshow(spectra[:, frequency_slice].T.real, extent=[0, 32, 0, 4.2], aspect='auto')
    axis[int(n/5),n%5].set_title(specs[n].split('/')[-1][4:22].replace('nuc-1H', ''))
figure.supxlabel('Average')
figure.supylabel('Chemical shift (ppm)')

In [ ]:
def freq_corr_all(mrs, ref, method='sr'):
    freq_corr = []
    for i in range(mrs.shape[0]):
        corr = suspect.processing.frequency_correction.spectral_registration(first, mrs[i,:], method=method)
        freq_corr.append(mrs[i,:].adjust_frequency(corr[0]))
    return mrs.inherit(np.array(freq_corr))


In [ ]:
 # Average

In [ ]:
w_avg = [np.mean(i,axis=0) for i in w_freq_corr]

In [ ]:
der_path = '/home/orco/data/MacroMols/MRS/MacroMols_3T/derivatives/suspect_preprocessing'
# Save as raw
l = list()
for n, i in enumerate(w_avg):
    l = water[n].split('/')[-1].split('_')
    filename = water[n].split('/')[-1].replace('.dat', 'avg.raw')
    out_path = os.path.join(der_path, l[0], 'mrs')
    os.makedirs(out_path, exist_ok=True)
    out = os.path.join(out_path, filename)
    print(out)
    np.savez(out, mrsdata=i)
    suspect.io.lcmodel.save_raw(out, i)

In [ ]:
%%time
averaged = [np.mean(i,axis=0) for i in freq_corr2]

In [ ]:
der_path = '/home/orco/data/MacroMols/MRS/MacroMols_3T/derivatives/suspect_preprocessing'
# Save as raw
l = list()
for n, i in enumerate(averaged):
    l = specs[n].split('/')[-1].split('_')
    filename = specs[n].split('/')[-1].replace('.dat', 'avg.raw')
    out_path = os.path.join(der_path, l[0], 'mrs')
    os.makedirs(out_path, exist_ok=True)
    out = os.path.join(out_path, filename)
    print(out)
    np.savez(out, mrsdata=i)
    suspect.io.lcmodel.save_raw(out, i)

In [ ]:
#EC

In [ ]:
def eddy_correct(spec, water):
    eddy_current = np.unwrap(np.angle(water))
    ec_smooth = suspect.processing.denoising.sliding_gaussian(eddy_current, 32)
    ecc = np.exp(-1j * ec_smooth)
    ecc_data = spec * ecc
    return ecc_data

In [ ]:
avg_ecc = list()
for n in range(len(averaged)):
    eddy_current = np.unwrap(np.angle(w_avg[n]))
    ec_smooth = suspect.processing.denoising.sliding_gaussian(eddy_current, 32)
    ecc = np.exp(-1j * ec_smooth)
    ec_data = averaged[n] * ecc
    avg_ecc.append(ec_data)

In [ ]:
for n,i in enumerate(avg_ecc):
    plt.plot(i.frequency_axis_ppm(), i.spectrum().real, label=n)
plt.xlim([5,0])
#plt.ylim([-2e-4,5e-4])
plt.axvline(4.7)
plt.legend()

In [ ]:
first = avg_ecc[0]

In [ ]:
plt.plot(first.frequency_axis_ppm(), first.adjust_phase(0).adjust_frequency(0).spectrum().real)
plt.plot(first.frequency_axis_ppm(), first.adjust_phase(0,-0.3e-3).adjust_frequency(6).spectrum().real)
plt.xlim([4.2,0])
#plt.ylim([-100,500])
plt.axvline(1.99)
plt.axvline(4.7)
plt.axvline(3.027)
plt.axvline(0.94)
plt.axhline(0)

In [ ]:
first= first.adjust_phase(0,-0.3e-3).adjust_frequency(6)

In [ ]:
arr_ecc = np.array(avg_ecc)
print(arr_ecc.shape)
arr_ecc = avg_ecc[0].inherit(arr_ecc)

In [ ]:
sr_spectra = arr_ecc.spectrum()
frequency_slice = sr_spectra.slice_ppm(4.2, 0)
plt.imshow(sr_spectra[:, frequency_slice].T.real, extent=[0, 13, 0, 4.2], aspect='auto')

In [ ]:
def freq_corr_all(mrs, ref, method='sr'):
    freq_corr = []
    for i in range(mrs.shape[0]):
        corr = suspect.processing.frequency_correction.spectral_registration(first, mrs[i,:], method=method)
        freq_corr.append(mrs[i,:].adjust_frequency(corr[0]))
    return mrs.inherit(np.array(freq_corr))

In [ ]:
avg_ecc_c = list()
for i in avg_ecc:
    corr = suspect.processing.frequency_correction.spectral_registration(first, i)
    avg_ecc_c.append(i.adjust_frequency(corr[0]))

In [ ]:
arr_ecc = np.array(avg_ecc_c)
print(arr_ecc.shape)
arr_ecc = avg_ecc_c[0].inherit(arr_ecc)
sr_spectra = arr_ecc.spectrum()
frequency_slice = sr_spectra.slice_ppm(4.2, 0)
plt.imshow(sr_spectra[:, frequency_slice].T.real, extent=[0, 13, 0, 4.2], aspect='auto')

In [ ]:
# Save as npz
l = list()
for n, i in enumerate(avg_ecc_c):
    l = specs[n].split('/')[-1].split('_')
    filename = specs[n].split('/')[-1].replace('.dat', '_avg_ecc.raw')
    out_path = os.path.join(der_path, l[0], 'mrs')
    os.makedirs(out_path, exist_ok=True)
    out = os.path.join(out_path, filename)
    print(out)
    np.savez(out, mrsdata=i)
    suspect.io.lcmodel.save_raw(out, i)

In [ ]:
# HLSVD

In [ ]:
water_list = list()
dry_list = list()
for spec in avg_ecc_c:
    components = suspect.processing.water_suppression.hsvd(spec, 30)
    water_components = [component for component in components if component["frequency"] < 45]
    water_fid = spec.inherit(suspect.processing.water_suppression.construct_fid(water_components, spec.time_axis()))
    dry_fid = spec - water_fid
    water_list.append(water_fid)
    dry_list.append(dry_fid)

In [ ]:
for i in water_list:
    plt.plot(i.frequency_axis_ppm(), 
             i.spectrum().real)
    plt.xlim([5,0])
    #plt.ylim([-2e-4,5e-4])
    #plt.axvline(4.7)

In [ ]:
i=0
plt.plot(dry_list[0].frequency_axis_ppm(), dry_list[i].adjust_phase(0,0).spectrum().real)
plt.plot(dry_list[0].frequency_axis_ppm(), avg_ecc_c[i].adjust_phase(0,0).spectrum().real)
plt.xlim([9,0])
#plt.ylim([-2e-4,5e-4])
plt.axvline(4.7)

In [ ]:
for i in dry_list:
    plt.plot(i.frequency_axis_ppm(), 
             i.spectrum().real)
    plt.xlim([5,0])
    #plt.ylim([-2e-4,5e-4])
    plt.axvline(1.99)
    plt.axvline(3.02)

In [ ]:
# Save as npz
l = list()
for n, i in enumerate(dry_list):
    l = specs[n].split('/')[-1].split('_')
    filename = specs[n].split('/')[-1].replace('.dat', 'avg_ecc_hlsvd.raw')
    out_path = os.path.join(der_path, l[0], 'mrs')
    os.makedirs(out_path, exist_ok=True)
    out = os.path.join(out_path, filename)
    print(out)
    np.savez(out, mrsdata=i)
    suspect.io.lcmodel.save_raw(out, i)